Sorry for making another collab notebook but alas I'm too tired and too close to the end to optmize this process

In [ ]:
%%capture
!git clone https://github.com/dpcomp-org/dpcomp_core.git

In [ ]:
%%capture
!pip install diffprivlib

In [ ]:
# Remove default module
%%capture
%cd dpcomp_core
!rm -r dpcomp_core

In [ ]:
%%capture
!wget https://dev.universalities.com/dpcomp_core.zip

In [ ]:
%%capture
!unzip dpcomp_core.zip

In [ ]:
# Add requirements so pip does the version matching right
!echo diffprivlib >> resources/requirements.txt
!echo pandas >> resources/requirements.txt

In [ ]:
%%capture
!pip install -r resources/requirements.txt

In [ ]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 117 kB/s 


In [ ]:
%%capture
!apt-get install swig

In [ ]:
%%capture
%cd dpcomp_core/algorithm/dawa/
!bash setup.sh
%cd ../../../

In [ ]:
import os
os.getcwd()

'/content/dpcomp_core'

In [ ]:
%cd dpcomp_core/algorithm/ahp/
!bash setup.sh
%cd ../../../

/content/dpcomp_core/dpcomp_core/algorithm/ahp
rm: cannot remove 'ahp_fast.c': No such file or directory
rm: cannot remove '*.so': No such file or directory
Compiling lib/ahp_fast.pyx because it changed.
[1/1] Cythonizing lib/ahp_fast.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/dpcomp_core/dpcomp_core/algorithm/ahp/lib/ahp_fast.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'ahp_fast' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/lib
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c lib/ahp_fast.c

In [ ]:
from __future__ import division
from __future__ import print_function
from past.utils import old_div
from dpcomp_core.algorithm import dawa
from dpcomp_core.algorithm import AG
from dpcomp_core.algorithm import ahp
from dpcomp_core import dataset
from dpcomp_core import workload
import numpy as np

In [ ]:
def runDAWA(data, bin, epsilon):
  domain = (bin,bin)
  epsilon = epsilon


  seed = 1
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                         domain_shape=domain, 
                         size=size, 
                         seed=seed)

  a = dawa.dawa2D_engine()
  H, xedges, yedges = np.histogram2d(data['x'], data['y'], bins = bin)
  H = H.T
  x_hat = a.Run(w, H, epsilon, seed)
  return x_hat

In [ ]:
def runAHP(data, bin, epsilon):
  domain = (bin,bin)

  seed = 1
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                        domain_shape=domain, 
                        size=size, 
                        seed=seed)

  a = ahp.ahpND_engine(ratio = 0.85, eta = 0.35)
  H, xedges, yedges = np.histogram2d(data['x'], data['y'], bins = bin)
  H = H.T
  x_hat = a.Run(w, H, epsilon, seed)
  return x_hat

In [ ]:
def runAGrid(data, bin, epsilon):
  domain = (bin,bin)
  seed = 1
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                        domain_shape=domain, 
                        size=size, 
                        seed=seed)

  a = AG.AG_engine(c = 10, c2 = 5, alpha = 0.5)
  H, xedges, yedges = np.histogram2d(data['x'], data['y'], bins = bin)
  H = H.T
  x_hat = a.Run(w, H, epsilon, seed)
  return x_hat

In [ ]:
from diffprivlib.tools import histogram2d
def runGeoTruncated(data, bin, epsilon):
  newList = histogram2d(data['x'],data['y'],epsilon,bin, density = False)
  x_hat = newList[0].T
  return x_hat


In [ ]:
from diffprivlib.mechanisms.laplace import Laplace

from sys import maxsize
def histogramdd(sample, epsilon, bins, range=None, weights=None, density=None, accountant=None, **unused_args):

    hist, bin_edges = np.histogramdd(sample, bins=bins, range=range, normed=None, weights=weights, density=None)

    dp_mech = Laplace().set_epsilon(epsilon).set_sensitivity(1)
    dp_hist = np.zeros_like(hist)
    iterator = np.nditer(hist, flags=['multi_index'])

    while not iterator.finished:
        dp_hist[iterator.multi_index] = dp_mech.randomise(int(iterator[0]))
        iterator.iternext()

    dp_hist = dp_hist.astype(float, casting='safe')

    if density:
        # calculate the probability density function
        dims = len(dp_hist.shape)
        dp_hist_sum = dp_hist.sum()
        for i in np.arange(dims):
            shape = np.ones(dims, int)
            shape[i] = dp_hist.shape[i]
            # noinspection PyUnresolvedReferences
            dp_hist = dp_hist / np.diff(bin_edges[i]).reshape(shape)

        if dp_hist_sum > 0:
            dp_hist /= dp_hist_sum

  

    return dp_hist, bin_edges


In [ ]:
def laplaceHistogram(array_x, array_y, epsilon, bins, range=None, weights=None, density=None, accountant=None,
                **unused_args):

    try:
        num_bins = len(bins)
    except TypeError:
        num_bins = 1

    if num_bins not in (1, 2):
        xedges = yedges = np.asarray(bins)
        bins = [xedges, yedges]

    hist, edges = histogramdd([array_x, array_y], epsilon=epsilon, bins=bins, range=range, weights=weights,
                              density=density, accountant=accountant)
    
    return hist, edges[0], edges[1]


In [ ]:
def runLaplace(data, bin, epsilon):
  newList = laplaceHistogram(data['x'],data['y'],epsilon,bin, density = False)
  x_hat = newList[0].T
  return x_hat


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

# Teaser


Please remove the colorbars and the numbers and ticks

In [ ]:
df = pd.read_csv('/dataFull/14.csv')
H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = 64)
epsilon = .5
Original = H.T
AHP = runAHP(df, 64, epsilon)
DAWA = runDAWA(df, 64, epsilon)
LP = runLaplace(df, 64, epsilon)

charts = [Original, AHP, DAWA, LP]

min = np.amin([np.amin(c) for c in charts])
max = np.amax([np.amax(c) for c in charts])
centercolor = np.amax([abs(min), abs(max)])

cbarlocs = [.21, .47, .733, .994]

for consistent in [ False]:

  fig = make_subplots(
      rows=1, cols=4,
      subplot_titles=("Original","AHP", "DAWA", "Laplace"),
      horizontal_spacing = 0.01
      )
  
  cbarlocs = [.21, .475, .736, .997]

  for i, data in enumerate(charts):

    fig.add_trace(
        go.Heatmap(
            z=data,
            colorscale=(
                ['rgb(153, 31, 31)','rgb(255,255,255)','rgb(3, 86, 94)'] if consistent else [[0, 'rgb(255,255,255)'], [1, 'rgb(3, 86, 94)']]),
            zmin=(-centercolor if consistent else None), zmax=(centercolor if consistent else None),  # use global min/max if consistent
            colorbar=None
            ),
        row=1, col=i+1
    )
  fig.update_traces(showscale=False)
  fig.update_layout(height=500, width=1500,
                    #title_text=("Algorithm Comparison with Consistent Coloring" if consistent else "Algorithm Comparison with Varied Coloring"),
                    margin=dict(l=20, r=20, t=40, b=20),
                    #font=dict(size=60),
                    coloraxis_showscale=False
                    )
  
  fig.update_xaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
  fig.update_yaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
  fig.write_image("teaser.svg")
  fig.show()

/content/dpcomp_core/dpcomp_core/query_nd_union.py:95: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  array[ndRangeUnion.slice(r)] += r.wgt


# Fig 2 - APQE

I think a third image including the original would work well. I think this works better than the other one. I think I may also not have correct data for the other one.

In [ ]:
def getL2(H, x_hat, bins):
  seed = 1
  domain = (bins,bins)
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                         domain_shape=domain, 
                         size=size, 
                         seed=seed)


  diff = w.evaluate(H) - w.evaluate(x_hat)
  # print('Per Query Average Absolute Error:', old_div(np.linalg.norm(diff,1), float(diff.size)))
  return(old_div(np.linalg.norm(diff,1), float(diff.size)))

In [ ]:
df = pd.read_csv('/dataFull/14.csv')
H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = 64)
H = H.T
f = None
s = None
L2Goal = np.linspace(.3, .6, 10)
for e in L2Goal:
  dawaAnswer = runDAWA(df, 64, e)
  geoAnswer = runLaplace(df, 64, e)
  first = getL2(H, dawaAnswer, 64)
  second = getL2(H, geoAnswer, 64)
  f = dawaAnswer
  s = geoAnswer
  print(first, second)
  if abs(first-second)<2:
    print(first)
    print(second)
    f = dawaAnswer
    s = geoAnswer
    print(e)
    break
  



/content/dpcomp_core/dpcomp_core/query_nd_union.py:95: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



25.8828 30.074614232981258
24.1964 25.5033472451382
24.1964
25.5033472451382
0.3333333333333333


In [ ]:
charts = [f, s]
# min = np.amin([np.amin(c) for c in charts])
# max = np.amax([np.amax(c) for c in charts])

fig = make_subplots(
      rows=1, cols=2,
      subplot_titles=("DAWA - APQE: 23.022", "Laplace - APQE:  22.467"),
      horizontal_spacing = 0.01
   
      )
fig.update_annotations(font = dict(size = 30, color = 'black'))

for i, data in enumerate(charts):

  fig.add_trace(
      go.Heatmap(
          z=data,
          colorscale=([[0, 'rgb(255,255,255)'], [1, 'rgb(3, 86, 94)']]),
          #zmin=min, zmax=max, # use global min/max # JANE TODO: Colorscales per chart
          #xgap=1, ygap=1,
          colorbar=None,
          showscale=False
          ),
      row=1, col=i+1
  )

fig.update_layout(height=600, width=1000,
                  margin=dict(l=20, r=20, t=40, b=20),
                  #title_text=("Algorithms with the same APQE have different visual utility")
                  )
fig.update_xaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.write_image("apqe.svg")

fig.show()

# Fig 3-5
all Good


#Fig 6 - epsilon



Again don't think it needs the color bar nor does it need the axis numbers


In [ ]:
df = pd.read_csv('/pilotData/9.csv')
H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = 64)
H = H.T
charts = [H, runAHP(df, 64, .3), runAHP(df, 64, .2), runAHP(df, 64, .1)]
min = np.amin([np.amin(c) for c in charts])# use global min/max
max = np.amax([np.amax(c) for c in charts])

fig = make_subplots(
      rows=1, cols=4,
      subplot_titles=("Original", "Epsilon: 0.3", "Epsilon: 0.2", "Epsilon: 0.1"),
      horizontal_spacing = 0.01
      )

for i, data in enumerate(charts):

  fig.add_trace(
      go.Heatmap(
          z=data,
          colorscale=([[0, 'rgb(255,255,255)'], [1, 'rgb(3, 86, 94)']]),
          zmin=min, zmax=max, # use global min/max
          #xgap=1, ygap=1,
          colorbar=None,
          showscale=False
          ),
      row=1, col=i+1
  )

fig.update_layout(height=400, width=1200,
                  margin=dict(l=20, r=20, t=40, b=20),
                  #title_text=("Variations in Epsilon")
                  )
fig.update_xaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.write_image("epsilon.svg")
fig.show()

# Fig 7 - bin size

I think we need to make the color scale different for each chart here because otherwise it's an unfair comparison as the 128 can get lost because the bin counts get really white.

In [ ]:
df = pd.read_csv('/pilotData/9.csv')
H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = 64)
H = H.T
charts = [H, runGeoTruncated(df, 32, .1), runGeoTruncated(df, 64, .1), runGeoTruncated(df, 128, .1)]
min = np.amin([np.amin(c) for c in charts])# use global min/max
max = np.amax([np.amax(c) for c in charts])

fig = make_subplots(
      rows=1, cols=4,
      subplot_titles=("Original", "Bins: 32x32", "Bins: 64x64", "Bins: 128x128"),
      horizontal_spacing = 0.01
      )

for i, data in enumerate(charts):
  min = np.amin(data)
  max = np.amax(data)
  fig.add_trace(
      go.Heatmap(
          z=data,
          colorscale=([[0, 'rgb(255,255,255)'], [1, 'rgb(3, 86, 94)']]),
          zmin=min, zmax=max, # use global min/max
          #xgap=1, ygap=1
          ),
      row=1, col=i+1
  )
fig.update_traces(showscale=False)
fig.update_layout(height=400, width=1200,
                  margin=dict(l=20, r=20, t=40, b=20),
                  #title_text=("Variations in Epsilon")
                  )

fig.update_xaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.write_image("bins.svg")
fig.show()

/usr/local/lib/python3.7/dist-packages/diffprivlib/tools/histograms.py:229: PrivacyLeakWarning:

Range parameter has not been specified (or has missing elements). Falling back to taking range from the data.
 To ensure differential privacy, and no additional privacy leakage, the range must be specified for each dimension independently of the data (i.e., using domain knowledge).

/usr/local/lib/python3.7/dist-packages/diffprivlib/tools/histograms.py:229: PrivacyLeakWarning:

Range parameter has not been specified (or has missing elements). Falling back to taking range from the data.
 To ensure differential privacy, and no additional privacy leakage, the range must be specified for each dimension independently of the data (i.e., using domain knowledge).

/usr/local/lib/python3.7/dist-packages/diffprivlib/tools/histograms.py:229: PrivacyLeakWarning:

Range parameter has not been specified (or has missing elements). Falling back to taking range from the data.
 To ensure differential privacy

# Results Figure Chart Scoring

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lpanavas/PrivateScaterplotData/master/pilotData/3.csv')
H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = 64)
H = H.T
charts = [runGeoTruncated(df, 32, .01), runGeoTruncated(df, 32, .08), runGeoTruncated(df, 32, .2), runGeoTruncated(df, 32, .7)]
# min = np.amin([np.amin(c) for c in charts])# use global min/max
# max = np.amax([np.amax(c) for c in charts])

fig = make_subplots(
      rows=1, cols=4,
      subplot_titles=("0", "1", "2", "3"),
      horizontal_spacing = 0.01,
     
      )

fig.update_annotations(font_size=35)

for i, data in enumerate(charts):
  min = np.amin(data)
  max = np.amax(data)
  fig.add_trace(
      go.Heatmap(
          z=data,
          colorscale=([[0, 'rgb(255,255,255)'], [1, 'rgb(3, 86, 94)']]),
          
          zmin=min, zmax=max, # use global min/max
          #xgap=1, ygap=1
          colorbar = None
          ),
      row=1, col=i+1
  )
fig.update_traces(showscale=False)
fig.update_layout(height=400, width=1200,
                  #title_text=("Ratings"),
                  margin=dict(l=20, r=20, t=40, b=20),
                  
                  )
fig.update_xaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.write_image("ratings.svg")
fig.show()

/usr/local/lib/python3.7/dist-packages/diffprivlib/tools/histograms.py:229: PrivacyLeakWarning:

Range parameter has not been specified (or has missing elements). Falling back to taking range from the data.
 To ensure differential privacy, and no additional privacy leakage, the range must be specified for each dimension independently of the data (i.e., using domain knowledge).

/usr/local/lib/python3.7/dist-packages/diffprivlib/tools/histograms.py:229: PrivacyLeakWarning:

Range parameter has not been specified (or has missing elements). Falling back to taking range from the data.
 To ensure differential privacy, and no additional privacy leakage, the range must be specified for each dimension independently of the data (i.e., using domain knowledge).

/usr/local/lib/python3.7/dist-packages/diffprivlib/tools/histograms.py:229: PrivacyLeakWarning:

Range parameter has not been specified (or has missing elements). Falling back to taking range from the data.
 To ensure differential privacy

# Recommendation System


In [ ]:
df = pd.read_csv('/fullData/14.csv')
H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = 64)
epsilon = .5
# Original = H.T
AHP = runAHP(df, 32, epsilon)
DAWA = runDAWA(df, 32, epsilon)
AGrid = runAGrid(df, 32, epsilon)
Geo = runGeoTruncated(df,32, epsilon)
LP = runLaplace(df, 32, epsilon)

charts = [AHP, DAWA, AGrid, LP, Geo]

min = np.amin([np.amin(c) for c in charts])
max = np.amax([np.amax(c) for c in charts])
centercolor = np.amax([abs(min), abs(max)])

# cbarlocs = [.21, .47, .733, .994]

for consistent in [ False]:

  fig = make_subplots(
      rows=1, cols=5,
      subplot_titles=("AHP", "DAWA", "AGrid", "Laplace", "Geometric"),
      horizontal_spacing = 0.01
      )
  fig.update_annotations(font = dict(size = 30, color = 'black'))
  
  # cbarlocs = [.21, .475, .736, .997]

  for i, data in enumerate(charts):

    fig.add_trace(
        go.Heatmap(
            z=data,
            colorscale=(
                ['rgb(153, 31, 31)','rgb(255,255,255)','rgb(3, 86, 94)'] if consistent else [[0, 'rgb(255,255,255)'], [1, 'rgb(3, 86, 94)']]),
            zmin=(-centercolor if consistent else None), zmax=(centercolor if consistent else None),  # use global min/max if consistent
            colorbar=None
            ),
        row=1, col=i+1
    )
  fig.update_traces(showscale=False)
  fig.update_layout(height=350, width=1500,
                    #title_text=("Algorithm Comparison with Consistent Coloring" if consistent else "Algorithm Comparison with Varied Coloring"),
                    margin=dict(l=20, r=20, t=40, b=20),
                    #font=dict(size=60),
                    coloraxis_showscale=False
                    )
  
  fig.update_xaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
  fig.update_yaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
  fig.write_image("useCase.svg")
  fig.show()

/usr/local/lib/python3.7/dist-packages/diffprivlib/tools/histograms.py:229: PrivacyLeakWarning:

Range parameter has not been specified (or has missing elements). Falling back to taking range from the data.
 To ensure differential privacy, and no additional privacy leakage, the range must be specified for each dimension independently of the data (i.e., using domain knowledge).



In [ ]:
df = pd.read_csv('/fullData/14.csv')
H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = 64)
H = H.T
charts = [runDAWA(df, 32, .5), runDAWA(df, 32, .1), runDAWA(df, 32, .05), runDAWA(df, 32, .01)]
min = np.amin([np.amin(c) for c in charts])# use global min/max
max = np.amax([np.amax(c) for c in charts])

fig = make_subplots(
      rows=1, cols=4,
      subplot_titles=("Epsilon: 0.5", "Epsilon: 0.1", "Epsilon: 0.05", "Epsilon: 0.01"),
      horizontal_spacing = 0.01
      )
fig.update_annotations(font = dict(size = 30, color = 'black'))


for i, data in enumerate(charts):

  fig.add_trace(
      go.Heatmap(
          z=data,
          colorscale=(
                ['rgb(153, 31, 31)','rgb(255,255,255)','rgb(3, 86, 94)'] if consistent else [[0, 'rgb(255,255,255)'], [1, 'rgb(3, 86, 94)']]),
            zmin=(-centercolor if consistent else None), zmax=(centercolor if consistent else None),  # use global min/max if consistent
          # zmin=min, zmax=max, # use global min/max
          #xgap=1, ygap=1,
          colorbar=None,
          showscale=False
          ),
      row=1, col=i+1
  )

fig.update_layout(height=400, width=1200,
                  margin=dict(l=20, r=20, t=40, b=20),
                  #title_text=("Variations in Epsilon")
                  )
fig.update_xaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.write_image("epsilonUseCase.svg")
fig.show()

In [ ]:
!pip install pytorch-msssim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as colors
import plotly.express as px
from PIL import Image
# import tensorflow as tf
import torch
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM


In [ ]:


df = pd.read_csv('/fullData/14.csv')
fig = go.Figure()
fig.add_trace(go.Scatter(mode = 'markers', x=df['x'], y=df['y'],
                         marker=dict(
            color='rgb(3, 86, 94)',
            size=7,
            opacity=0.25),
            
            ))

fig.update_layout(height=600, width=600,
             
                  
                  )
fig.update_xaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_yaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_layout(
    plot_bgcolor='rgb(255,255,255)'
)
fig.write_image(str(0) +'.png')
fig.write_image("original_graph.svg")
fig.show()


In [ ]:
bins = list(range(10, 55, 5))
print(len(bins))

for b in bins:
  # newList = histogram2d(df['x'],df['y'],.3,b, density = False)
  x_hat = runDAWA(df, b, .5)
  fig = go.Figure()
  fig.add_trace(go.Heatmap(
            z=x_hat,
            colorscale=([0, 'rgb(255,255,255)'], [1, 'rgb(3, 86, 94)']),
        
            ))
  fig.update_layout(height=600, width=600,
                    #title_text=("Algorithm Comparison with Consistent Coloring" if consistent else "Algorithm Comparison with Varied Coloring"),
                    margin=dict(l=20, r=20, t=40, b=20),
                    #font=dict(size=60),
                    coloraxis_showscale=False
                    )
  fig.update_traces(showscale=False)

  fig.update_xaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
  fig.update_yaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
  fig.update_layout(
      plot_bgcolor='rgb(255,255,255)'
  )
  fig.write_image(str(b) +'.png')
  
  # fig.show()



9


/content/dpcomp_core/dpcomp_core/query_nd_union.py:95: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



In [ ]:
msDict = {}
max  = 0
for b in bins:
  img = Image.open('0.png')
  X = np.array(img).astype(np.float32)

  img = Image.open(str(b)+'.png')

  Y = np.array(img).astype(np.float32)

  img_torch = torch.from_numpy(X).unsqueeze(0).permute(0, 3, 1, 2)  # 1, C, H, W
  img_noise_torch = torch.from_numpy(Y).unsqueeze(0).permute(0, 3, 1, 2)
  msssim_torch = ms_ssim(img_noise_torch, img_torch, win_size=11, data_range=255)

  val = msssim_torch.item()
  if val > max:
    max = val
  msDict[b] = round(val, 3)

In [ ]:
msDict

{10: 0.673,
 15: 0.676,
 20: 0.68,
 25: 0.67,
 30: 0.674,
 35: 0.675,
 40: 0.659,
 45: 0.652,
 50: 0.665}

In [ ]:
from math import floor
fig = make_subplots(
    rows=3, cols=3,
    horizontal_spacing = 0.01,
    vertical_spacing = 0.05
    )
for i, (key, value) in enumerate(msDict.items()):
    x_hat = runDAWA(df, key, .5)
    fig.add_trace(go.Heatmap(
            z=x_hat,
            colorscale=([0, 'rgb(255,255,255)'], [1, 'rgb(3, 86, 94)']),
            ),
        row=floor(i/3)+1, col=floor(i%3)+1
    )
    fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.12, showarrow=False,
                   text=("Bins: " + str(key)+ ", MS-SSIM: "+ str(value)), row=floor(i/3)+1, col=floor(i%3)+1)
    fig.update_annotations(font = dict(size = 30, color = 'black'))

    fig.update_traces(showscale=False)
    # fig.update_traces(height=200, width=200 )
fig.update_layout(height=1200, width=1200,
                    #title_text=("Algorithm Comparison with Consistent Coloring" if consistent else "Algorithm Comparison with Varied Coloring"),
                    # margin=dict(l=20, r=20, t=40, b=20),
                    #font=dict(size=60),
                    coloraxis_showscale=False
                    )
fig.update_xaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_yaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_layout(
    plot_bgcolor='rgb(255,255,255)'
)
fig.write_image("binsUseCase.svg")
fig.show()
  

In [ ]:
import matplotlib
from matplotlib import cm
from matplotlib.colors import ListedColormap
# magma_cmap = matplotlib.cm.get_cmap('magma')
N = 256
yellow = np.ones((N, 4))
yellow[:, 0] = np.linspace(3/256, 1, N) # R = 255
yellow[:, 1] = np.linspace(86/256, 1, N) # G = 232
yellow[:, 2] = np.linspace(94/256, 1, N)  # B = 11
yellow_cmp = ListedColormap(yellow)
yellow_cmp = ListedColormap(yellow_cmp.colors[::-1])




  # fig.upd

In [ ]:
def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        # print(np.array(cmap[k])*255)
        C = list(map(np.uint8, np.array(cmap[k])*255))
        # print(C)
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

In [ ]:
gammas = np.linspace(.1,1.5,9)
# gammas = [1,2,3,4,5,6,7,8,9]


for g in gammas:
  g = round(g, 3)
  norm=matplotlib.colors.PowerNorm(gamma=g, vmin=0, vmax=255)

  magma_rgb = []
  for i in range(0, 255):
        k = matplotlib.colors.colorConverter.to_rgb(yellow_cmp(norm(i)))
        magma_rgb.append(k)
  magma = matplotlib_to_plotly(magma_rgb, 255)

  x_hat = runDAWA(df, 20, .5)
  fig = go.Figure()
  fig.add_trace(go.Heatmap(
            z=x_hat,
            colorscale=(magma),
        
            ))
  fig.update_layout(height=600, width=600,
                    #title_text=("Algorithm Comparison with Consistent Coloring" if consistent else "Algorithm Comparison with Varied Coloring"),
                    margin=dict(l=20, r=20, t=40, b=20),
                    #font=dict(size=60),
                    coloraxis_showscale=False
                    )
  fig.update_traces(showscale=False)

  fig.update_xaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
  fig.update_yaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
  fig.update_layout(
      plot_bgcolor='rgb(255,255,255)'
  )
  fig.write_image(str(g) +'Gamma.png')



In [ ]:
gammaDict = {}
max  = 0
for g in gammas:
  g = round(g, 3)

  img = Image.open('0.png')
  X = np.array(img).astype(np.float32)

  img = Image.open(str(g)+'Gamma.png')

  Y = np.array(img).astype(np.float32)

  img_torch = torch.from_numpy(X).unsqueeze(0).permute(0, 3, 1, 2)  # 1, C, H, W
  img_noise_torch = torch.from_numpy(Y).unsqueeze(0).permute(0, 3, 1, 2)
  msssim_torch = ms_ssim(img_noise_torch, img_torch, win_size=11, data_range=255)

  val = msssim_torch.item()
  if val > max:
    max = val
  gammaDict[g] = round(val, 3)

gammaDict

{0.1: 0.581,
 0.275: 0.608,
 0.45: 0.633,
 0.625: 0.655,
 0.8: 0.67,
 0.975: 0.679,
 1.15: 0.685,
 1.325: 0.689,
 1.5: 0.691}

In [ ]:
from math import floor
fig = make_subplots(
    rows=3, cols=3,
    horizontal_spacing = 0.01,
    vertical_spacing = 0.05
    )
for i, (key, value) in enumerate(gammaDict.items()):
   
    g = round(g, 3)
    norm=matplotlib.colors.PowerNorm(gamma=key, vmin=0, vmax=255)

    magma_rgb = []
    for j in range(0, 255):
          k = matplotlib.colors.colorConverter.to_rgb(yellow_cmp(norm(j)))
          magma_rgb.append(k)
    magma = matplotlib_to_plotly(magma_rgb, 255)
    x_hat = runDAWA(df, 20, .5)
    fig.add_trace(go.Heatmap(
            z=x_hat,
            colorscale=(magma),
            ),
        row=floor(i/3)+1, col=floor(i%3)+1
    )
    fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.13, showarrow=False,
                   text=("Gamma: " + str(key)+ ", MS-SSIM: "+ str(value)), row=floor(i/3)+1, col=floor(i%3)+1)
    fig.update_annotations(font = dict(size = 23, color = 'black'))

    fig.update_traces(showscale=False)
    # fig.update_traces(height=200, width=200 )
fig.update_layout(height=1200, width=1200,
                    #title_text=("Algorithm Comparison with Consistent Coloring" if consistent else "Algorithm Comparison with Varied Coloring"),
                    # margin=dict(l=20, r=20, t=40, b=20),
                    #font=dict(size=60),
                    coloraxis_showscale=False
                    )
fig.update_xaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_yaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_layout(
    plot_bgcolor='rgb(255,255,255)'
)
fig.write_image("gammaUseCase.svg")
fig.show()

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lpanavas/PrivateScaterplotData/master/fullStudyDataForImages/14.csv')
fig = go.Figure()
fig.add_trace(go.Heatmap(
            z=x_hat,
            colorscale=(magma),
            ))

fig.update_layout(height=600, width=600,
             
                  
                  )
fig.update_traces(showscale=False)
fig.update_xaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_yaxes(showline=True, title = '', linewidth=1, linecolor='darkgrey', mirror=True, showticklabels=False)
fig.update_layout(
    plot_bgcolor='rgb(255,255,255)'
)
fig.write_image('final.png')
fig.write_image("finalGraphUseCase.svg")
fig.show()